In [2]:
import lseg.data as ld
import pandas as pd
from lseg.data.content import news
from IPython.display import HTML

In [1]:
from langchain_ollama.chat_models import ChatOllama

# ChatNVIDIA.get_available_models()
llm = ChatOllama(model="llama3.1:8b-instruct-q8_0", temperature=0,)


In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x13ff5cc50 {name='workspace'}>

In [10]:
definition = news.top_news.hierarchy.Definition()
response = definition.get_data()
energy_top_news_id = response.data.hierarchy['Commodities']['Energy'].top_news_id
definition = news.top_news.Definition(energy_top_news_id)
news_response = definition.get_data()
print(news_response.data.df['storyId'].iloc(0)[0])

urn:newsml:reuters.com:20250930:nL3N3VH0SJ


In [26]:
#Retrieve the first top news story
storyId = news_response.data.df["storyId"].iloc(0)[0]
news_story = ld.news.get_story(storyId)
with open('news_story.html', 'w+') as f:
    f.write(news_story)
    
display(HTML(news_story))

In [27]:
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader('news_story.html')
docs = loader.load()
docs

[Document(metadata={'source': 'news_story.html', 'title': ''}, page_content='Exxon restructuring affects 3-4% of global workforceNo US job cuts planned, half in Canadian unit Imperial OilOil price slump prompts industry-wide job cuts, efficiency measuresAdds detail about layoffs across EU; background on EU corporate sustainability lawBy Sheila Dang and Vallari  SrivastavaHOUSTON, Sept 30 (Reuters) - Top U.S. oil major Exxon Mobil XOM.Nsaid on Tuesday it will lay off 2,000 workers globally, particularly in Canada and across the European Union, as part of a long-term restructuring plan that will affect about 3% to 4% of the company\'s workforce.About 1,200 positions will be cut in Norway and the EU by the end of 2027, Exxon said in a statement.There are no planned job cuts in the U.S., and the layoffs announced on Monday at Canadian oil producer Imperial OilIMO.TO, of which Exxon is a majority owner, account for about half of the broader cuts confirmed on Tuesday, a company spokesperson 

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("""Write a concise summary of the following: '{context}'""")

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

Here is a concise summary of the article:

Exxon Mobil will lay off 2,000 workers globally, about 3-4% of its workforce, as part of a restructuring plan. The layoffs will primarily affect Canada and the European Union, with no job cuts planned in the US. The company aims to improve efficiency by consolidating office locations and having employees work from the same locations. This move comes amid a slump in global crude oil prices and increased competition among energy companies. Exxon's CEO has also criticized an EU corporate sustainability law that threatens fines for environmental issues, which may have contributed to the decision to reduce its European presence.
